# Model Comparison

Constructing your language model application will likely involved choosing between many different options of prompts, models, and even chains to use. When doing so, you will want to compare these different options on different inputs in an easy, flexible, and intuitive way. 

LangChain provides the concept of a ModelLaboratory to test out and try different models.

In [1]:
%pip show langchain

Name: langchain
Version: 0.0.238
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /media/limcheekin/My Passport/langchain-playground/venv/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain import Modal, OpenAI
from langchain.model_laboratory import ModelLaboratory
import os

In [14]:
ct2_llm = Modal(
    endpoint_url=os.environ["LLAMA2_13B_CHAT_CT2_URL"], 
    sampling_temperature=0.0, 
    max_length=2042, 
    include_prompt_in_result=False)

ggml_llm = OpenAI(
    openai_api_base=os.environ["LLAMA2_13B_CHAT_GGML_URL"], 
    temperature=0.0, 
    max_tokens=2042)

llms = [ggml_llm, ct2_llm]


sampling_temperature was transferred to model_kwargs.
                    Please confirm that sampling_temperature is what you intended.
max_length was transferred to model_kwargs.
                    Please confirm that max_length is what you intended.
include_prompt_in_result was transferred to model_kwargs.
                    Please confirm that include_prompt_in_result is what you intended.


In [15]:
# REF: https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat/blob/main/model.py#L24
DEFAULT_SYSTEM_PROMPT = "You are a helpful assistant.\nYou will try to answer user questions, but don't make up the answer if you don't have the answer.\nYou will complete tasks by following user instructions."
def get_prompt(message: str, chat_history: list[tuple[str, str]],
               system_prompt: str) -> str:
    texts = [f'[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']
    for user_input, response in chat_history:
        texts.append(f'{user_input.strip()} [/INST] {response.strip()} </s><s> [INST] ')
    texts.append(f'{message.strip()} [/INST]')
    return ''.join(texts)

In [16]:
%%time
print(ggml_llm(get_prompt("What is AI?", [], DEFAULT_SYSTEM_PROMPT)))

  Sure thing! I'd be happy to help answer your question about AI.

AI stands for artificial intelligence, which refers to the ability of machines or computer programs to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. AI systems use algorithms and data to make predictions, classify objects, and generate insights based on patterns they detect.

There are many different types of AI, including:

1. Narrow or weak AI, which is designed to perform a specific task, such as facial recognition or language translation.
2. General or strong AI, which is designed to perform any intellectual task that a human can.
3. Superintelligence, which is significantly more intelligent than the best human minds.
4. Artificial general intelligence (AGI), which is a hypothetical AI system that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks, similar to human intelligence.
AI has many applications in va

In [17]:
%%time
print(ct2_llm("What is AI?"))

KeyboardInterrupt: 

In [ ]:
model_lab = ModelLaboratory.from_llms(llms)

In [ ]:
model_lab.compare("What color is a flamingo?")

Input:
What color is a flamingo?

Modal
Params: {'endpoint_url': 'https://limcheekin--flan-t5-xl-ct2-fastapi-app.modal.run/', 'model_kwargs': {}}
pink

Modal
Params: {'endpoint_url': 'https://limcheekin--flan-alpaca-gpt4-xl-ct2-fastapi-app.modal.run/', 'model_kwargs': {}}
Flamingos are typically pink in color.



In [21]:
prompt = PromptTemplate(template="What is the capital of {state}?", input_variables=["state"])
model_lab_with_prompt = ModelLaboratory.from_llms(llms, prompt=prompt)

In [22]:
model_lab_with_prompt.compare("New York")

Input:
New York

Modal
Params: {'endpoint_url': 'https://limcheekin--flan-t5-xl-ct2-fastapi-app.modal.run/', 'model_kwargs': {}}
albany

Modal
Params: {'endpoint_url': 'https://limcheekin--flan-alpaca-gpt4-xl-ct2-fastapi-app.modal.run/', 'model_kwargs': {}}
The capital of New York is Albany.



In [5]:
context = """\
1) Flutter is a framework for building cross-platform applications
that uses the Dart programming language.
To understand some differences between programming with Dart
and programming with Javascript, 
see Learning Dart as a JavaScript Developer.
2) Web FAQ

Performance FAQ

What is Flutter?

Flutter is Google’s portable UI toolkit for crafting beautiful,
natively compiled applications for mobile, web,
and desktop from a single codebase.
Flutter works with existing code,
is used by developers and organizations around
the world, and is free and open source.

Who is Flutter for?

For users, Flutter makes beautiful apps come to life.
3) Flutter is a cross-platform UI toolkit that is designed to allow code reuse
across operating systems such as iOS and Android, while also allowing
applications to interface directly with underlying platform services. The goal
is to enable developers to deliver high-performance apps that feel natural on
different platforms, embracing differences where they exist while sharing as
much code as possible.
"""
question = "What is Flutter?"
prompt = f"Answer based on context:\n\n{context}\n\n{question}"
model_lab.compare(prompt)


Input:
Answer based on context:

1) Flutter is a framework for building cross-platform applications
that uses the Dart programming language.
To understand some differences between programming with Dart
and programming with Javascript, 
see Learning Dart as a JavaScript Developer.
2) Web FAQ

Performance FAQ

What is Flutter?

Flutter is Google’s portable UI toolkit for crafting beautiful,
natively compiled applications for mobile, web,
and desktop from a single codebase.
Flutter works with existing code,
is used by developers and organizations around
the world, and is free and open source.

Who is Flutter for?

For users, Flutter makes beautiful apps come to life.
3) Flutter is a cross-platform UI toolkit that is designed to allow code reuse
across operating systems such as iOS and Android, while also allowing
applications to interface directly with underlying platform services. The goal
is to enable developers to deliver high-performance apps that feel natural on
different platforms, 

In [6]:
prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
model_lab.compare(prompt)
selected_qa_prompt=prompt

Input:
Context: 1) Flutter is a framework for building cross-platform applications
that uses the Dart programming language.
To understand some differences between programming with Dart
and programming with Javascript, 
see Learning Dart as a JavaScript Developer.
2) Web FAQ

Performance FAQ

What is Flutter?

Flutter is Google’s portable UI toolkit for crafting beautiful,
natively compiled applications for mobile, web,
and desktop from a single codebase.
Flutter works with existing code,
is used by developers and organizations around
the world, and is free and open source.

Who is Flutter for?

For users, Flutter makes beautiful apps come to life.
3) Flutter is a cross-platform UI toolkit that is designed to allow code reuse
across operating systems such as iOS and Android, while also allowing
applications to interface directly with underlying platform services. The goal
is to enable developers to deliver high-performance apps that feel natural on
different platforms, embracing differe

In [7]:
dialogue = """\
Human: Have you talked to Miro?
AI: No, not really, I've never had an opportunity
Human: yes, he's so interesting
Human: told me the story of his father coming from Albania to the US in the early 1990s
AI: really, I had no idea he is Albanian
Human: he is, he speaks only Albanian with his parents
AI: fascinating, where does he come from in Albania?
Human: from the seacoast
Human: Duress I believe, he told me they are not from Tirana
AI: what else did he tell you?
Human: That they left kind of illegally
Human: it was a big mess and extreme poverty everywhere
Human: then suddenly the border was open and they just left
Human: people were boarding available ships, whatever, just to get out of there
Human: he showed me some pictures, like <file_photo>
AI: insane
Human: yes, and his father was among the people
AI: scary but interesting
Human: very!
"""
prompts = [
        f"{dialogue}\n\nBriefly summarize that dialogue.", 
        f"Here is a dialogue:\n{dialogue}\n\nWrite a short summary!",
        f"Dialogue:\n{dialogue}\n\nWhat is a summary of this dialogue?",
        f"{dialogue}\n\nWhat was that dialogue about, in two sentences or less?",
        f"Here is a dialogue:\n{dialogue}\n\nWhat were they talking about?",
        f"Dialogue:\n{dialogue}\nWhat were the main points in that conversation?",
        f"Dialogue:\n{dialogue}\nWhat was going on in that conversation?"
]

for prompt in prompts:
    model_lab.compare(prompt)

Input:
Human: Have you talked to Miro?
AI: No, not really, I've never had an opportunity
Human: yes, he's so interesting
Human: told me the story of his father coming from Albania to the US in the early 1990s
AI: really, I had no idea he is Albanian
Human: he is, he speaks only Albanian with his parents
AI: fascinating, where does he come from in Albania?
Human: from the seacoast
Human: Duress I believe, he told me they are not from Tirana
AI: what else did he tell you?
Human: That they left kind of illegally
Human: it was a big mess and extreme poverty everywhere
Human: then suddenly the border was open and they just left
Human: people were boarding available ships, whatever, just to get out of there
Human: he showed me some pictures, like <file_photo>
AI: insane
Human: yes, and his father was among the people
AI: scary but interesting
Human: very!


Briefly summarize that dialogue.

Modal
Params: {'endpoint_url': 'https://limcheekin--flan-t5-xl-ct2-fastapi-app.modal.run/', 'model_kwa

In [9]:
question = "Did he tell you anything else happening there?"
prompt = f"""Your name is Flutty, you can answer any question about Flutter but nothing else. 
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question.
Conversation:
{dialogue}
Follow up question: {question}
Standalone question:"""
prompt

"Your name is Flutty, you can answer any question about Flutter but nothing else. \nGiven the following conversation and a follow up question,\nrephrase the follow up question to be a standalone question.\nConversation:\nHuman: Have you talked to Miro?\nAI: No, not really, I've never had an opportunity\nHuman: yes, he's so interesting\nHuman: told me the story of his father coming from Albania to the US in the early 1990s\nAI: really, I had no idea he is Albanian\nHuman: he is, he speaks only Albanian with his parents\nAI: fascinating, where does he come from in Albania?\nHuman: from the seacoast\nHuman: Duress I believe, he told me they are not from Tirana\nAI: what else did he tell you?\nHuman: That they left kind of illegally\nHuman: it was a big mess and extreme poverty everywhere\nHuman: then suddenly the border was open and they just left\nHuman: people were boarding available ships, whatever, just to get out of there\nHuman: he showed me some pictures, like <file_photo>\nAI: ins

In [10]:
model_lab.compare(prompt)

Input:
Your name is Flutty, you can answer any question about Flutter but nothing else. 
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question.
Conversation:
Human: Have you talked to Miro?
AI: No, not really, I've never had an opportunity
Human: yes, he's so interesting
Human: told me the story of his father coming from Albania to the US in the early 1990s
AI: really, I had no idea he is Albanian
Human: he is, he speaks only Albanian with his parents
AI: fascinating, where does he come from in Albania?
Human: from the seacoast
Human: Duress I believe, he told me they are not from Tirana
AI: what else did he tell you?
Human: That they left kind of illegally
Human: it was a big mess and extreme poverty everywhere
Human: then suddenly the border was open and they just left
Human: people were boarding available ships, whatever, just to get out of there
Human: he showed me some pictures, like <file_photo>
AI: insane
Human: ye

In [11]:
dialogue = ""
question = "Tell me about yourself."
prompt = f"""Your name is Flutty, you can answer any question about Flutter but nothing else. 
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question.
Conversation:
{dialogue}
Follow up question: {question}
Standalone question:"""
prompt

'Your name is Flutty, you can answer any question about Flutter but nothing else. \nGiven the following conversation and a follow up question,\nrephrase the follow up question to be a standalone question.\nConversation:\n\nFollow up question: Tell me about yourself.\nStandalone question:'

In [12]:
model_lab.compare(prompt)


Input:
Your name is Flutty, you can answer any question about Flutter but nothing else. 
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question.
Conversation:

Follow up question: Tell me about yourself.
Standalone question:

Modal
Params: {'endpoint_url': 'https://limcheekin--flan-t5-xl-ct2-fastapi-app.modal.run/', 'model_kwargs': {'max_length': 256}}
What is your name?

Modal
Params: {'endpoint_url': 'https://limcheekin--flan-alpaca-gpt4-xl-ct2-fastapi-app.modal.run/', 'model_kwargs': {'max_length': 256}}
Flutty, can you tell me about yourself?

Modal
Params: {'endpoint_url': 'https://limcheekin--falcon-7b-instruct-ct2-fastapi-app.modal.run/', 'model_kwargs': {'sampling_temperature': 0.0, 'max_length': 128, 'include_prompt_in_result': False}}
 What is your name?

Modal
Params: {'endpoint_url': 'https://limcheekin--mpt-7b-instruct-ct2-fastapi-app.modal.run/', 'model_kwargs': {'sampling_temperature': 0.0, 'max_length': 128